# Graphing the bias when equal mixing fails to hold

This notebook uses functions to simulate the distribution of one- and two-vehicle crashes under different assumptions about driver behavior, including:
1. The relative risk of a drinking driving causing a fatal two-vehicle crash (theta);
1. The relative risk of a drinking driving causing a fatal one-vehicle crash (lambda);
1. The probability of two drinking-drivers interacting in excess of their share of road users;
1. The relative ability of non-drinking drivers to evade potentially fatal errors committed by other road users.
1. The true prevalence of drinking (BAC>0) and legally-impaired (BAC>.08) driving equals the observed prevalence in the National Roadside Survey.

Using the generated distribution, prevalence is estimated assuming drivers:
1. Obey equal and independent mixing; 
1. Fatal crashes arise when one driver makes an error;
1. All driver types have the same probability of evading a fatal crash when the other driver has committed a potentially fatal error.

<p> These are assumptions in the baseline Levitt and Porter (2001) approach and imply the crash distribution follows the multinomial distribution. 
    
<p>Estimates of prevalence under different values of the excess interaction probability are calculated at the observed NRS estimates and plotted. These plots include the excess interaction probability that would be necessary to reconcile the prevelances estimates reported in Dunn and Tefft (AMAR, forthcoming) based on the LP approach assuming the NRS estimates were unbiased.
    

<p><b>Run All Cells</b> to generate an interactive widget at the bottom of the notebook.

## Set preliminaries

In [1]:
# Import modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Define underlying parameters for simulation

drivers=100000
theta_ref=.000001
evade_ref=.1

style = {'description_width': 'initial'}

## Function that constructs crash distribution and estimates parameters from multinomial distribution

In [2]:
def multinom_estimator(p, x, m, theta, evade_drink):
    
    ## convert percentage to decimal
    P=p/100
    X=x/100
    mix=m/100
    
    ## Number of drivers and ratio
    N_ref=drivers*(1-P-X)
    N_drink=drivers*P
    N=N_drink/N_ref

    ## Risk of a drinking driver causing a two-vehicle crash
    theta_drink=theta_ref*theta

    ## Risk of causing a single-vehicle crash
    lamb=theta
    lambda_ref=theta_ref
    lambda_drink=theta_drink

    # Calculate fatal error probability conditional on an interaction and crash distribution

    f_dd=2*(1-evade_drink)*theta_drink
    f_ds=(1-evade_drink)*theta_drink+(1-evade_ref)*theta_ref
    f_ss=2*(1-evade_ref)*theta_ref
    

    ## Assumes that every driver has 2 interactions with every other driver

    A_dd=N_drink*(N_drink*f_dd*(1+mix))
    A_ss=N_ref*(N_ref*f_ss)
    A_ds=2*N_ref*N_drink*f_ds

    

    # Calculate parameters of mulinomial and solve for theta

    R=A_ds*A_ds/(A_dd*A_ss)

    theta_hat=((R-2) + np.sqrt(R*R - 4*R))/2
    N_hat = np.sqrt((A_dd/A_ss)/theta_hat)
    P_hat = (N_hat/(1+N_hat))*(1-X)

    return P_hat

## Function that calls estimation and plots estimated prevalence as equal mixing changes

In [3]:
def pi(impaired='Drinking',  mix_max=200, evade_drink=.1, theta=10):
    
    mix_array=np.arange(0,mix_max/100,.01)

    if impaired=='Drinking':
        # Get estimates from multinomial function given prevalence from NRS

        estimates_2007=np.array([multinom_estimator(12.4,0, mix, theta, evade_drink) for mix in np.arange(0,mix_max)])
        estimates_2013=np.array([multinom_estimator(8.3,0, mix, theta, evade_drink) for mix in np.arange(0,mix_max)])

        # Find index where estimates equal LP estimates

        idx_2007=np.argmin(np.abs(estimates_2007-.143))
        idx_2013=np.argmin(np.abs(estimates_2013-.15))

        fig, ax = plt.subplots()

        line_2007=ax.plot(mix_array, estimates_2007, label='2007: NRS prevalence=' + str(12.4)+'%')
        line_2013=ax.plot(mix_array, estimates_2013, label='2013: NRS prevalence=' + str(8.3)+'%')

        ax.set(xlabel='Excess probability of two drinking-drivers interacting', ylabel='Estimated prevalence',
               title='Bias in LP estimates')
        ax.grid()
        ax.legend()


        plt.text(mix_array[idx_2007],.143, str(100*mix_array[idx_2007])+'%',horizontalalignment='right')
        plt.text(mix_array[idx_2013],.15, str(100*mix_array[idx_2013])+'%',horizontalalignment='right')

    else:
        # Get estimates from multinomial function given prevalence from NRS

        estimates_1996=np.array([multinom_estimator(4.3,16.9-4.3, mix, theta, evade_drink) for mix in np.arange(0,mix_max)])
        estimates_2007=np.array([multinom_estimator(2.2,12.4-2.2, mix, theta, evade_drink) for mix in np.arange(0,mix_max)])
        estimates_2013=np.array([multinom_estimator(1.5,8.3-1.5, mix, theta, evade_drink) for mix in np.arange(0,mix_max)])

        # Find index where estimates equal LP estimates

        idx_1996=np.argmin(np.abs(estimates_1996-.104))
        idx_2007=np.argmin(np.abs(estimates_2007-.092))
        idx_2013=np.argmin(np.abs(estimates_2013-.099))

        fig, ax = plt.subplots()

        line_1996=ax.plot(mix_array, estimates_1996, label='1996: NRS prevalence=' + str(4.3)+'%')
        line_2007=ax.plot(mix_array, estimates_2007, label='2007: NRS prevalence=' + str(2.2)+'%')
        line_2013=ax.plot(mix_array, estimates_2013, label='2013: NRS prevalence=' + str(1.5)+'%')

        ax.set(xlabel='Excess probability of two legally-impaired drivers interacting', ylabel='Estimated prevalence from LP',
               title='Bias in LP estimates')
        ax.grid()
        ax.legend()

        plt.text(mix_array[idx_1996],.104, str(100*mix_array[idx_1996])+'%',horizontalalignment='right')
        plt.text(mix_array[idx_2007],.092, str(100*mix_array[idx_2007])+'%',horizontalalignment='right')
        plt.text(mix_array[idx_2013],.099, str(100*mix_array[idx_2013])+'%',horizontalalignment='right')

        
    plt.annotate('Notes: theta='+ str(theta) + '; mu=' +str((1-evade_drink)/(1-evade_ref)), (0,0), (0, -50), xycoords='axes fraction', textcoords='offset points', va='top')

    #fig.savefig("equal_mixing_drinking.png")
    plt.show()





## Interactive LP Widget

The default considers drinking versus non-drinking drivers.

<div class="alert alert-block alert-danger" >
    <p font color='black'>&#x25BA; If an error arises as you lower theta, provide a lower maximum excess probability. 
    <p font color='black'>&#x25BA; If the returned excess probability equals the maximum, increase the maximum.
</div>

In [4]:
interact(pi, 
         evade_drink=fixed(.1),
         impaired=widgets.RadioButtons(
             options=['Drinking', 'Legally impaired'],value='Drinking',description='Driver type:',disabled=False),
         mix_max=widgets.BoundedFloatText(
             min=0, max=1000, value=100,description='Maximum excess probability (%): ',disabled=False, style=style),
         theta=widgets.FloatSlider(
             min=4, max=25, step=.5, value=7)
        )

interactive(children=(RadioButtons(description='Driver type:', options=('Drinking', 'Legally impaired'), value…

<function __main__.pi(impaired='Drinking', mix_max=200, evade_drink=0.1, theta=10)>

## References
1. Dunn RA & Tefft NW (2021). Drinking-and-driving in the United States from 1983-2017: comparing survey and model-based estimates of prevalence. <i>Analytic Methods in Accident Research</i>, forthcoming.
1. Levitt SD & Porter J (2001). How dangerous are drinking-drivers? <i>Journal of Political Economy</i>, 109(6):1198-1237.
1. Ramirez, A, Berning, A., Kelley-Baker, T., Lacey, J. H., Yao, J., Tippetts, A. S., … & Compton, R. (2016, December). 2013–2014 National Roadside Study of Alcohol and Drug Use by Drivers: Alcohol Results (Report No. DOT HS 812 362). Washington, DC: National Highway Traffic Safety Administration.